Maximum inner product search using Asymmetric Locality Sensitive Hashing

In [7]:
import os
import numpy as np
import pandas as pd
from scipy import sparse

Executed on a smaller dataset at present


In [8]:
# Run this only after downloading the small or large dataset and make sure the path is set

# This is in case of small dataset
dataset = "datasets" + os.path.sep + "ml-latest-small"

# This is in case of large dataset
# dataset = "datasets"+os.path.sep+"ml-latest"

# Name of the csv file
name = "ratings.csv"

# Read the csv file and get the appropriate column IDs
ratings_df = pd.read_csv(dataset + os.path.sep + name, names= ["UserID", "MovieID", "Rating", "Timestamp"], header=0)

# Converting to numbers from strings
ratings_df["UserID"] = pd.to_numeric(ratings_df["UserID"], errors='ignore')
ratings_df["MovieID"] = pd.to_numeric(ratings_df["MovieID"], errors='ignore')
ratings_df["Rating"] = pd.to_numeric(ratings_df["Rating"], errors='ignore')
ratings_df["Timestamp"] = pd.to_numeric(ratings_df["Timestamp"], errors='ignore')

In [9]:
# Checking the head of the df created

ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [10]:
# Take 80% of the shuffled data as the actual data and the rest as query data

#ratings_data_df =  ratings_df[0:int(0.8*len(ratings_df))]
#ratings_query_df = ratings_df[int(0.8*len(ratings_df)):]

#print ratings_data_df.shape
#print ratings_query_df.shape

In [12]:
# Create the matrix of the rating of movies with the userIDs, fill with 0 for the missing values 

R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
UserID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The below part is the SVD part on data after splitting the dataset into 80:20 for data and query

In [13]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [14]:
# Divided the dataset into test(Query) and Train(Data Points)
#convert the rankings dataframe to sparse matrix
sR = sparse.csr_matrix(R_demeaned)
TRAIN_SIZE = 0.80
# Create boolean mask
# np.random creates a vector of random values between 0 and 1
# Those values are filtered to create a binary mask
msk = np.random.rand(sR.shape[0],sR.shape[1]) < TRAIN_SIZE
r = np.zeros(sR.shape)
print (msk.shape)
 
train_ratings = sR.copy()
test_ratings = sR.copy()
#mask itself is random
train_ratings[msk] = r[msk]
test_ratings[~msk] = r[~msk] # inverse of boolean mask
test_r = np.array(test_ratings)
train_r = np.array(train_ratings)

(671, 9066)


In [26]:
type(test_ratings)

scipy.sparse.csr.csr_matrix

In [30]:
from scipy.sparse.linalg import svds
#Ratings to SVD
U, sigma, Vt = svds(train_ratings, k = 20)

In [28]:
sigma = np.diag(sigma)

In [29]:
sigma

array([[ 43.74050605,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,  43.91903553,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,  44.75574386,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,  